In [42]:
library('igraph')
library('Matrix')
library('pracma')

create_transition_matrix = function (g){
    
    # WARNING: make sure your graph is connected (you might input GCC of your graph)
    
    vs = V(g)
    n = vcount(g)
    adj = as_adjacency_matrix(g)
    adj[diag(rowSums(adj) == 0)] = 1  # handle if the user is using the function for networks with isolated nodes by creating self-edges
    z = matrix(rowSums(adj, , 1))
    
    transition_matrix = adj / repmat(z, 1, n)  # normalize to get probabilities
    
    return(transition_matrix)
}

random_walk = function (g, num_steps, start_node, transition_matrix = NULL){
    if(is.null(transition_matrix))
        transition_matrix = create_transition_matrix(g)
    vert = seq_len(0)
    vs <- start_node
    start <- start_node
    for(i in 1:num_steps){
        #fprintf('Step %d: %d\n', i, v)  # COMMENT THIS
        PMF = transition_matrix[vs, ]
        v = sample(1:vcount(g), 1, prob = PMF)
        vert <- c(vert, v)
        vs <- v
    }
    return(vert)
}

In [51]:
random_walk_tele = function (g, num_steps, start_node, transition_matrix = NULL){
    if(is.null(transition_matrix))
        transition_matrix = create_transition_matrix(g)
    vert = seq_len(0)
    vs <- start_node
    start <- start_node
    prob = c(0,1) 
    for(i in 1:num_steps){
        #fprintf('Step %d: %d\n', i, v)  # COMMENT THIS
        jump = sample(prob, 1, prob=c(0.85,0.15))
        PMF = NULL
        if (jump == 0) {
            PMF = transition_matrix[vs, ]
        }
        v = sample(1:vcount(g), 1, prob = PMF)
        vert <- c(vert, v)
        vs <- v
    }
    return(vert)
}

In [52]:
set.seed(1)
rm_graph = barabasi.game(1000, m=4, directed=T)
start = sample(1:vcount(rm_graph), 1)
vert = random_walk_tele(rm_graph, 1000, start)

In [55]:
print(vert)

   [1] 322  23  13   2   1   1   1   1   1   1   1   1   1   1   1   1   1   1
  [19]   1   1   1   1   1 796 459 266   4   1   1   1   1   1 647 598  37   2
  [37]   1   1   1   1 314   1   1   1   1 978  16   3   1   1   1   1  31   2
  [55]   1 397   3 808  15   6  56   2 378   2   1   1   1   1   1   1   1   1
  [73]   1   1   1   1   1   1 781   3   2 334 482  93   6   2   1 648  22   2
  [91]   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1 960   2   1
 [109] 114   2   1   1   1   1 257 980  62   1   1   1   1 862 290   4 393   3
 [127]   2   1   1   1   1 992 176 895   5   2   1   1   1   1   1   1   1   1
 [145]   1   1   1   1   1   1   1   1 610  65   4 903   9   4   1   1   1   1
 [163]   1 768  22   2   1   1   1   1 358   2   1   1  70   6   2   1   1   1
 [181]   1   1   1   1 665   2   1   1   1   1   1 144   1 659  30   8   3   2
 [199]   1   1   1   1 176  12   3 818 656  71   3   1   1 239   4   2   1 425
 [217]   2   1   1   1   1   1   1   1   1   1   1  

In [57]:
freq = function (vertex_sequence, num_vertex, num_steps) {
    name = names(table(vertex_sequence))
    count = as.numeric(table(vertex_sequence))
    result = seq_len(0)
    for (i in 1:num_vertex){
        if(i %in% name){
            result = c(result, count[i]/(num_steps + 1))
        }
        else
            result = c(result, 0)
    }
    result[is.na(result)] = 0
    return(result)
}
    
random_walk_personalize = function (g, num_steps, start_node, transition_matrix = NULL, PageRank=NULL){
    if(is.null(transition_matrix))
        transition_matrix = create_transition_matrix(g)
    vert = seq_len(0)
    vs <- start_node
    start <- start_node
    prob = c(0,1) 
    for(i in 1:num_steps){
        jump = sample(prob, 1, prob=c(0.85,0.15))
        PMF = transition_matrix[vs, ]
        if (jump == 1) {
            PMF = PageRank
        }
        v = sample(1:vcount(g), 1, prob = PMF)
        vert <- c(vert, v)
        vs <- v
    }
    return(vert)
}

In [58]:
PageRank = freq(vert,1000, 1000)

In [62]:
vert1 = random_walk_personalize(rm_graph, 1000, start, PageRank=PageRank)

In [63]:
vert1

[1] 342   2   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1 168
  [19]   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
  [37]   1   1   1   1   1   1   1   1   1   1   1   2   1   1   1   6   2   1
  [55]   1   1   1   2   1   1   1   1   1   1   1   1   1   1   1   1   1   1
  [73]   1   1   1   1   3   2  59  45   3   1   1   1   1   1   2   3   2   1
  [91]   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [109]   1   1   1   1   1   1  62   5   4   1   1   1   1   1   1   1   1   1
 [127]   1   1   1   1   1   1   1   1   1   1   3   1   1   5   1   1   1   1
 [145]   1   1   1   1   1   4   2   1   1   1   1   1   1   1   1   1   1   1
 [163]   1   1   1   1   1   1   1   1   1   1   1   1   1   3   2   1   9   5
 [181]   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [199]   1   1   1   1   1   1   1   1   1   3   1   1   1   1   1   1   1   1
 [217]   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   5
 [235]   4   1   1   1   1   1   1   1   1   1   1   1   1   1   2   1   1   1
 [253]   1   2   1   1   1   1   1   1   1   1   1   1   1   1   1   1  12   5
 [271]   2   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [289]   1   1   1   1   1   2   1   1   1   1   1   1   1   1   1   1   1   1
 [307]   1   1 172 118 102  51   3   1   1   1   2   5   4   3   1   9   3   2
 [325]   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [343]   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [361]   1   1   1   1   1   1   1   1   1   1   1   1   6   1   1   1   1 102
 [379]  51   4   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [397]   1   2   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [415]   1   1   1   1   1   1   1   1   1   1   1   1   4   2   1   1   1   1
 [433]   6   4   2   1   1   1   1   1   1   1   1   1   1   2   1   1   1   1
 [451]   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [469]   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [487]   1   1   1   1   1   1   1   1   1   1   1   1   1 188   9   4   2   1
 [505]  23   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [523]   1  82   2   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [541]   1   1   1   1   1   1   1   1   1   1   3   1   1   1   1   1   1   1
 [559]   2   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [577]   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [595]   1   1   1   1   1   1   1 170  94   3   2   1   1   1   1   1   1   1
 [613] 144  94  30   4   2   1   1   1   1   1   1   1   1   1   1   1   1   1
 [631]   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [649]   1   1   1   1   1   1   1   2   1   1   1   1   1   1   1   1   1   1
 [667]   1   1   1   1   1   1   1   1   1   1  34   5   3   2   1   1   1   1
 [685]   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [703]   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [721]   1   1   1   3   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [739]   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [757]   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [775]   1   4   3   1   1   1   1 176  12   5   3   2   1   1   1   1   1   1
 [793] 204  14   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [811]   1   1   1   1   3   2   1   1   1   1   1   1   1   1   1 188   6   4
 [829]   1   1   1   1   1   3   1  34   3   1   1   1   1   1   1   1   1   1
 [847]   1   1   1   1   1   1   1   1   1   1   3   2 144   1   1   1   1   5
 [865]   4   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [883]   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 [901]   1   1   1   1   1   1   1   1   1   1   1   